# Imports

In [1]:
from numpy.random import seed
from tensorflow.random import set_seed

seed(88)
set_seed(404)

In [2]:
import os
import numpy as np
import tensorflow as tf

from time import strftime
from PIL import Image

# Constants

In [3]:
# Data paths
X_TRAIN_PATH = 'MNIST/digit_xtrain.csv'
X_TEST_PATH = 'MNIST/digit_xtest.csv'
Y_TRAIN_PATH = 'MNIST/digit_ytrain.csv'
Y_TEST_PATH = 'MNIST/digit_ytest.csv'

# Nr. of Digits
NR_CLASSES = 10

# Validation Dataset
VALIDATION_SIZE = 10_000

# Nr. of Inputs
TOTAL_INPUTS = 28 * 28 * 1

# Logging Paths
LOGGING_PATH = 'tensorboard_mnist_digit_logs/'

# Load Dataset

In [4]:
y_train = np.loadtxt(Y_TRAIN_PATH, delimiter=',', dtype=int)

In [5]:
y_test = np.loadtxt(Y_TEST_PATH, delimiter=',', dtype=int)

In [6]:
x_train = np.loadtxt(X_TRAIN_PATH, delimiter=',', dtype=int)

In [7]:
x_test = np.loadtxt(X_TEST_PATH, delimiter=',', dtype=int)

# Exploring Data

In [8]:
x_train.shape

(60000, 784)

In [9]:
y_train.shape

(60000,)

# Pre-Processing Data

In [10]:
x_train, x_test = x_train / 255, x_test / 255

### Convert Target-values to One-Hot Encoding

In [11]:
y_train = np.eye(NR_CLASSES)[y_train]

In [12]:
y_test = np.eye(NR_CLASSES)[y_test]

### Create Validation Datset from Training Data

In [13]:
y_val = y_train[:VALIDATION_SIZE]
x_val = x_train[:VALIDATION_SIZE]
y_train = y_train[VALIDATION_SIZE:]
x_train = x_train[VALIDATION_SIZE:]

# Setup Tensorflow Graph

In [14]:
tf.compat.v1.disable_eager_execution()

In [15]:
# Inputs
X = tf.compat.v1.placeholder(dtype=tf.float32, shape=[None, TOTAL_INPUTS], name='X')

In [16]:
# Output
Y = tf.compat.v1.placeholder(dtype=tf.float32, shape=[None, NR_CLASSES], name='Y')

## Neural Network Architecture

#### Hyperparametres

In [17]:
nr_epochs = 18
learning_rate = 1e-3   #0.001

nr_hidden_layer1 = 512
nr_hidden_layer2 = 64

#### First Hidden Layer

In [18]:
with tf.compat.v1.name_scope('Hidden_Layer_1'):
    
    # weights
    initial_weight1 = tf.random.truncated_normal(shape=[TOTAL_INPUTS, nr_hidden_layer1], stddev=0.1, seed=42)
    weight1 = tf.Variable(initial_weight1, name='weight_1')

    # bias
    initial_bias1 = tf.constant(value=0.0, shape=[nr_hidden_layer1])  # shape = (nr_hidden_layer2,)
    bias1 = tf.Variable(initial_bias1, name='bias_1')

    layer1_in = tf.matmul(X, weight1) + bias1
    layer1_out = tf.nn.relu(features=layer1_in)
    
    tf.compat.v1.summary.histogram('weights', weight1)
    tf.compat.v1.summary.histogram('bias', bias1)

#### Second Hidden Layer

In [19]:
with tf.compat.v1.name_scope('Hidden_Layer_2'):    
    
    # weights
    initial_weight2 = tf.random.truncated_normal(shape=[nr_hidden_layer1, nr_hidden_layer2], stddev=0.1, seed=42)
    weight2 = tf.Variable(initial_weight2, name='weight_2')

    # bias
    initial_bias2 = tf.constant(value=0.0, shape= [nr_hidden_layer2])   # shape = (nr_hidden_layer2,)
    bias2 = tf.Variable(initial_bias2, name='bias_2')

    layer2_in = tf.matmul(layer1_out, weight2) + bias2
    layer2_out = tf.nn.relu(layer2_in)
    
    tf.compat.v1.summary.histogram('weights', weight2)
    tf.compat.v1.summary.histogram('bias', bias2)

#### Output Layer

In [20]:
with tf.compat.v1.name_scope('Output_Layer'):    
    
    # weights
    initial_weight3 = tf.random.truncated_normal(shape=[nr_hidden_layer2, NR_CLASSES], stddev=0.1, seed=42)
    weight3 = tf.Variable(initial_weight3, name='weight_3')

    # bias
    initial_bias3 = tf.constant(value=0.0, shape= [NR_CLASSES])   
    bias3 = tf.Variable(initial_bias3, name='bias_3')

    layer3_in = tf.matmul(layer2_out, weight3) + bias3
    output = tf.nn.softmax(layer3_in)
    
    tf.compat.v1.summary.histogram('weights', weight3)
    tf.compat.v1.summary.histogram('bias', bias3)

#### DropOut Layer (Optional) 

In [21]:
# # DO Layer Setup
# dropout_layer = tf.compat.v1.nn.dropout(layer1_out, keep_prob=0.8, name='DropOut_Layer')

# # Changes to Second Layer
# layer2_in = tf.matmul(dropout_out, weight2) + bias2
# layer2_out = tf.nn.relu(layer2_in)

# Tensorboard Setup

In [22]:
# Folder for Tensorboard
model_name = f'{nr_hidden_layer1}-{nr_hidden_layer2} LR{learning_rate} E{nr_epochs}'
folder_name = f'{model_name} at {strftime("%H %M")}'
directory = os.path.join(LOGGING_PATH, folder_name)
try: 
    os.makedirs(directory)
except FileExistsError:
    pass

#### Tenseroard link:

In [23]:
#tensorboard --logdir=C:\Users\momoh\DataScienceProjects\ML\HANDWRITTEN_DIGITS_CLASSIFICATION\tensorboard_mnist_digit_logs

# Loss, Optimisation & Metrics

####  Defining Loss Function

In [24]:
with tf.compat.v1.name_scope('Loss_Calculation'):    
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=output))

####  Defining Optimizer

In [25]:
with tf.compat.v1.name_scope('Optimizer'):
    optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=learning_rate)
    train_step = optimizer.minimize(loss=loss)

#### Accuracy Metric

In [26]:
with tf.compat.v1.name_scope('Accuracy_Calculation'):
    model_prediction = tf.argmax(Y, axis=1, name='Prediction')
    correct_pred = tf.equal(model_prediction, tf.argmax(output, axis=1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [27]:
with tf.compat.v1.name_scope('Performance'):    
    tf.compat.v1.summary.scalar('accuracy', accuracy)
    tf.compat.v1.summary.scalar('loss', loss)

#### Check Input Images in Tensorboard

In [28]:
with tf.compat.v1.name_scope('Show_Image'):
    x_image = tf.reshape(X, [-1, 28, 28, 1])
    tf.compat.v1.summary.image('Image_Input', x_image, max_outputs=4)

# Run Session

In [29]:
sess = tf.compat.v1.Session()

#### Setup Filewriter and Merge Summaries

In [30]:
merged_summary = tf.compat.v1.summary.merge_all()

train_writer = tf.compat.v1.summary.FileWriter(directory + '/train')
train_writer.add_graph(sess.graph)

validation_writer = tf.compat.v1.summary.FileWriter(directory + '/validate')

#### Initialize all Variables

In [31]:
init = tf.compat.v1.global_variables_initializer()
sess.run(init)

## Batching Data

In [32]:
size_of_batch = 1000

In [33]:
num_examples = y_train.shape[0]
nr_iterations = int(num_examples/size_of_batch)

idx_epochs = 0

In [34]:
def next_batch(batch_size, data, labels):
    
    global num_examples
    global idx_epochs
    
    start = idx_epochs
    idx_epochs += batch_size
    
    if idx_epochs > num_examples:
        start = 0
        end = batch_size
        
    end = idx_epochs
    
    return data[start:end], labels[start:end]

### Training Loop

In [35]:
%%time

for epoch in range(nr_epochs):
    
    # ============ Training ===============
    
    for i in range(nr_iterations):
        
        X_batch, Y_batch = next_batch(batch_size=size_of_batch, data=x_train, labels=y_train)
        feed_dictionary = {X:X_batch, Y:Y_batch}
        
        # Training the Model
        sess.run(fetches=train_step, feed_dict=feed_dictionary)
        
    # Training Accuracy 
    batch_accuracy = sess.run(fetches=[accuracy], feed_dict=feed_dictionary)
    
    # Training Summary
    s = sess.run(fetches=merged_summary, feed_dict=feed_dictionary)
    train_writer.add_summary(s, epoch)
        
    print(f'Epoch {epoch} \t| Training Accuracy : {batch_accuracy}')
    
    # =========== Validation ================
    
    # Validation Summary
    val_summary = sess.run(fetches=merged_summary, feed_dict={X:x_val, Y:y_val})
    validation_writer.add_summary(val_summary, epoch)
    
print('Finished Training.')

Epoch 0 	| Training Accuracy : [0.859]
Epoch 1 	| Training Accuracy : [0.85268]
Epoch 2 	| Training Accuracy : [0.8679]
Epoch 3 	| Training Accuracy : [0.8776]
Epoch 4 	| Training Accuracy : [0.8844]
Epoch 5 	| Training Accuracy : [0.89588]
Epoch 6 	| Training Accuracy : [0.98148]
Epoch 7 	| Training Accuracy : [0.98644]
Epoch 8 	| Training Accuracy : [0.98862]
Epoch 9 	| Training Accuracy : [0.98982]
Epoch 10 	| Training Accuracy : [0.99106]
Epoch 11 	| Training Accuracy : [0.99178]
Epoch 12 	| Training Accuracy : [0.9925]
Epoch 13 	| Training Accuracy : [0.99304]
Epoch 14 	| Training Accuracy : [0.9933]
Epoch 15 	| Training Accuracy : [0.99346]
Epoch 16 	| Training Accuracy : [0.9937]
Epoch 17 	| Training Accuracy : [0.994]
Finished Training.
Wall time: 6min 43s


# Saving the Model

In [37]:
inputs = {'X': X}
outputs = {'Accuracy_Calculation/Prediction': model_prediction}

try:
    tf.compat.v1.saved_model.simple_save(sess, f'SavedModel-{model_name}', inputs, outputs)
except AssertionError:
    pass

# Making a Prediction

In [38]:
img = Image.open('MNIST/test_img.png')
bw = img.convert('L')
img_array = np.invert(bw)
test_img = np.ravel(img_array)

In [39]:
img

In [40]:
prediction = sess.run(fetches=tf.argmax(output, axis=1), feed_dict={X:[test_img]})

In [41]:
print('Image prediction is {}'.format(prediction))

Image prediction is [2]


# Test and Evaluation

In [42]:
test_accuracy = sess.run(fetches=accuracy, feed_dict={X:x_test, Y:y_test})
print('Test Dataset Prediction Accuracy is {:.2%}'.format(test_accuracy))

Test Dataset Prediction Accuracy is 97.64%


# Reset for the Next Run

In [43]:
train_writer.close()
validation_writer.close()
sess.close()
tf.compat.v1.reset_default_graph()